# Data-preprocessing notebook

    1) Create a new TARGET variable (0 user stayed, 1 user leaved). Check consitency and fix error (if exists)
    2) Drop all non-numerical variables (no time for encoding now. Leave it for another time)
    3) Create a new DELTA_LAST_INVOICE_LOAD: Time from the last money load in reference to current invoice
    4) Normalize datetime object in reference to account creation date
    5) Filter variables and users that have more than 500 NaNs or NaTs    
    
    Author: Nelson Fernandez Pinto
            @nfsrules
    
    

In [1]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

In [2]:
df = pd.read_csv('interview.csv')

In [22]:
# Check for nans and nats
df.groupby('ENTITY_ID').max().isna().sum().sort_values(ascending=False)

CHURN_DATE                             4793
LAST_FUNDRAISING_AMOUNT                4572
CHURNING_DATE                          4499
AMOUNT_SMI_REQ_EUR                     4289
NB_SMI_REQ                             4289
NUMBER_EXPORT                          3525
ACCOUNTING_EXPORTS_GENERATED           3525
MAIN_ENTITY                            3112
AMOUNT_EXP_CL_REQ_EUR                  3093
NB_EXP_CL_REQ                          3093
SUM_TICKETS_JIRA                       2827
NUMBER_OF_TEAMS_ACTIVE                 2774
SUB_SEGMENT                            2500
NPS                                    2374
AVG_SCORE                              2162
NB_SUBSCRIPTIONS_PAYMENTS              2080
AMOUNT_SUBSCRIPTIONS_PAYMENTS_EUR      2080
AMOUNT_PLASTIC_CARD_PAYMENTS_EUR       2027
NB_PLASTIC_CARD_PAYMENTS               2027
INDUSTRY                               1388
REQUESTER_LOGIN_PER_USER               1385
IS_FLAGSHIP                            1300
NB_SINGLE_PURCHASE_PAYMENTS     

In [17]:
def create_target(date):
    '''Check if churn rate date is NaT.
    Input: Datetime or number
    Returns:
        0 if date is NaT (still client)
        1 if date is not NaT (client left the company)
    '''
    return int(date != date)*-1+1


def isNaN(num):
    '''Check if num is NaN.
    Input: Number
    Returns:
        True if value is NaN
        False if value is not NaN
    
    '''
    return num != num


def check_churn_target(group, max_date):
    '''Complete missings churn dates.
    Input: Group (Pandas Dataframe of 1 user)
    Returns:
        Pandas dataframe of group with missing churn date added.
        New column TARGET (1: user leaved, 0: user stayerd)
    '''
    # Sort rows by date
    group.sort_values(by='MONTH', inplace=True)
    
    # Complete NaNs of churnig date (IN TOTAL 273 USERS)
    if not group.MONTH.max() == max_date: # the client already exited Spendesk
        if isNaN(group.CHURN_DATE.max()): # CHURN_DATE should not be NaN
            group.CHURN_DATE = group.CHURNING_DATE.max() + relativedelta(months=1)
    
    # Add target inplace to group
    group['TARGET'] = group.CHURN_DATE.apply(create_target)

    return group


def pre_processing_user(group, max_date):
    '''
    Create new variable 'DAYS_SINCE_LAST_LOAD'.
    Time in days between last time money was load to the platform
    in reference to last invoicing date.
    
    Normalize all dates in respecto to first invoice date.
    
    Input: User dataframe
    Return: User dataframe
    '''    
    # Sort rows by date
    group.sort_values(by='MONTH', inplace=True)
    
    # Complete NaNs of churnig date (IN TOTAL 273 USERS)
    if not group.MONTH.max() == max_date: # the client already exited Spendesk
        if isNaN(group.CHURN_DATE.max()): # CHURN_DATE should not be NaN
            group.CHURN_DATE = group.CHURNING_DATE.max() + relativedelta(months=1)
            
    # Add target (0: Stays in platform, 1: Leave the platform)
    group['TARGET'] = group.CHURN_DATE.apply(create_target)
    # Time from last money load last invoice 
    group['DAYS_SINCE_LAST_LOAD'] = (group.MONTH.max() - group.LAST_LOAD_DATE).dt.days
    # Number of days to the first money load
    group.FIRST_LOAD_DATE = (group.FIRST_LOAD_DATE - group.MONTH.iloc[0]).dt.days
    # Number of days to the last money load
    group.LAST_LOAD_DATE = (group.LAST_LOAD_DATE - group.MONTH.iloc[0]).dt.days
    # Normalize months
    group.MONTH = (group.MONTH - group.MONTH.iloc[0]).dt.days

    return group


def filter_nan(df_users, max_nan_allowed=500, verbose=True):
    '''Two-step NaN filtering.
    Input
        df: Pandas dataframe
        max_nan_allowed: Mas number of user with nans allowed by variable
        verbose: Print selected variables, default True
        
    Returns
        List of selected variables
        Pandas dataframe with selected variables
    '''
    # Get variable names that has less than 500 NaN users (first level filtering)
    var_names = df_users.columns.values
    
    # Create a boolean mask with variables that have less than 500 users with NaN values
    mask = (df_users.groupby('ENTITY_ID').max().isna().sum() < max_nan_allowed).values
    mask = np.insert(mask, 0, True, axis=0)
    selected_variables = var_names[mask]
    
    # Drop users that have NaNs on the pre-selected features (Second-level filtering)
    selected_users = df_users[selected_variables].groupby('ENTITY_ID').max().dropna().index 
    
    # Filter dataframe 
    df_filtered = df_users[selected_variables]
    filtered_df = pd.concat([df_filtered[df_filtered.ENTITY_ID == name] for name in selected_users])
    
    if verbose:
        print('Selected variables: ',selected_variables)
        print('The filtered dataset has {} and {} variables'.format(len(selected_users),
                                                            len(selected_variables)))
    return filtered_df


def pre_processing_dataset(df, max_nan_allowed=500, verbose=True):
    '''Pre-process dataframe:
    Add new variable DAYS_SINCE_LAST_LOAD: Time from last money load
    Normalize date variables respecting to first invoincing date.
    Drop all variables with more than max_nan_allowed users
    
    Input: Pandas Dataframe
    Returs: Pandas DataFrame
    '''
    # Convert date variables to datetime type
    df.MONTH = pd.to_datetime(df.MONTH)
    df.FIRST_LOAD_DATE = pd.to_datetime(df.FIRST_LOAD_DATE)
    df.CHURN_DATE = pd.to_datetime(df.CHURN_DATE) 
    df.CHURNING_DATE = pd.to_datetime(df.CHURNING_DATE) 

    # Assuming all dates are in the same datetime
    df.LAST_LOAD_DATE = pd.to_datetime(df.LAST_LOAD_DATE)
    df.LAST_LOAD_DATE = df.LAST_LOAD_DATE.apply(lambda x: x.replace(tzinfo=None)) # forget timezone info

    # The date of creation of the database is
    max_date = df.MONTH.max()
    # Pre-processing and add target to dataframe user per user
    df_users = df.groupby('ENTITY_ID').apply(lambda x: pre_processing_user(x, max_date))
    df_users = df_users.drop(columns=['ENTITY_ID']).reset_index().drop(columns=['level_1']) # back to original df shape
    
    # Filter Nans and NaTs
    df_filtered = filter_nan(df_users, 
                              max_nan_allowed=max_nan_allowed,
                              verbose=verbose)    
    return df_filtered



In [18]:
df_filtered = pre_processing_dataset(df)

Selected variables:  ['ENTITY_ID' 'MONTH' 'ORGANISATION_ID' 'COUNTRY' 'FIRST_LOAD_DATE'
 'LAST_LOAD_DATE' 'ORGANISATION_NAME' 'CS_OWNER' 'NAME' 'FTE_AGG'
 'AGG_INDUSTRY' 'AGE' 'SEGMENT' 'AVERAGE_TIME_PER_USER'
 'AMOUNT_PAY_PER_ACTIVE_USERS' 'COUNT_PAY_PER_ACTIVE_USERS'
 'ACTIVE_USERS_PER_FTE' 'USERS_PER_FTE' 'VAR_MRR_PER_FTE' 'VAR_MRR'
 'FIX_MRR' 'NB_ACTIVE_PLASTIC_CARDS' 'TARGET' 'DELTA_LAST_INVOICE_LOAD']
The filtered dataset has 4760 and 24 variables


In [12]:
# Save dataset
#df_filterd.to_csv('filtered_dataset.csv')